In [1]:
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir(next((root for root, _, files in os.walk(".") if "dsait4090_project_location" in files), "."))
    print(f'Google Colab: {os.getcwd()}')
except ImportError:
    print(f'Local: {os.getcwd()}')

Mounted at /content/drive
Google Colab: /content/drive/Othercomputers/macbook_pro/fact-checking


In [2]:
%load_ext autoreload
%autoreload

import json
import os
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

from src.common import get_device, read_data, save_data


device = get_device()

CUDA: 1, use Tesla T4


In [3]:
model_name = "numind/NuExtract-tiny-v1.5"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model.generation_config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

In [4]:
# template_statistical = """{
#     "Number": ""
# }"""
#
# template_interval = """{
#     "Beginning": "",
#     "End": "",
#     "Lower bound": "",
#     "Upper bound": ""
# }"""
#
# template_comparison = """{
#     "Number": ""
# }"""
#

template_temportal = """{
    "Duration": "",
    "Time": "",
    "Date": "",
    "Year": ""
}"""

template_numbers = """{
    "Numbers": []
}"""

In [ ]:
def predict_NuExtract(model, tokenizer, texts, template, batch_size=1, max_length=10_000, max_new_tokens=4_000):
    template = json.dumps(json.loads(template), indent=4)
    prompts = [f"""<|input|>\n### Template:\n{template}\n### Text:\n{text}\n\n<|output|>""" for text in texts]

    outputs = []
    with torch.no_grad():
        for i in tqdm(range(0, len(prompts), batch_size)):
            batch_prompts = prompts[i:i+batch_size]
            batch_encodings = tokenizer(batch_prompts, return_tensors="pt", truncation=True, padding=True, max_length=max_length).to(model.device)
            pred_ids = model.generate(**batch_encodings, max_new_tokens=max_new_tokens)
            output = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
            outputs += output

    results = []
    for output in outputs:
        try:
            results.append(output.split("<|output|>")[1])
        except:
            results.append(output)

    # return [output.split("<|output|>")[1] for output in outputs]
    return results

def extract_nums(split: str, category: str, template: str):

    claims = read_data(f'custom_decomposition/{split}_decomposed_flant5_predicted_type.json')
    claims = [claim for claim in claims if claim['taxonomy_label'].strip() == category]
    print(f'{split} {category}: {len(claims)}')

    claims_texts = [claim['claim'] for claim in claims]
    predictions = predict_NuExtract(model, tokenizer, claims_texts, template, batch_size=1)

    fail_counter = 0
    parsed_predictions = []
    for pred in predictions:
        try:
            parsed_predictions.append(json.loads(pred))
        except:
            parsed_predictions.append({})
            fail_counter += 1
            print(f'FAIL {fail_counter}\n{pred}')

    assert len(claims) == len(parsed_predictions)
    for claim, pred in zip(claims, parsed_predictions):
        claim['extracted_nums'] = pred

    save_data(f'custom_decomposition/{split}_extracted_{category}.json', claims)


extract_nums('train', 'statistical', template_numbers)
# extract_nums('train', 'interval', template_numbers)
# extract_nums('train', 'comparison', template_numbers)
# extract_nums('train', 'temporal', template_temportal)

# extract_nums('val', 'statistical', template_numbers)
# extract_nums('val', 'interval', template_numbers)
# extract_nums('val', 'comparison', template_numbers)
# extract_nums('val', 'temporal', template_temportal)

# extract_nums('test', 'statistical', template_numbers)
# extract_nums('test', 'interval', template_numbers)
# extract_nums('test', 'comparison', template_numbers)
# extract_nums('test', 'temporal', template_temportal)

train statistical: 4660


  0%|          | 0/4660 [00:00<?, ?it/s]